<a href="https://colab.research.google.com/github/ebitdad7/pegasus/blob/master/NLP_sum_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ebitdad7/pegasus.git
%cd pegasus
!export PYTHONPATH=.

Cloning into 'pegasus'...
remote: Enumerating objects: 171, done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (171/171), 353.09 KiB | 5.60 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/pegasus


In [2]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 1.1MB 9.1MB/s 
     |████████████████████████████████| 8.6MB 41.3MB/s 
     |████████████████████████████████| 1.4MB 50.5MB/s 
     |████████████████████████████████| 3.5MB 48.0MB/s 
     |████████████████████████████████| 411.0MB 37kB/s 
     |████████████████████████████████| 110.5MB 38kB/s 
     |████████████████████████████████| 307kB 49.2MB/s 
     |████████████████████████████████| 5.3MB 6.9MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 358kB 57.4MB/s 
     |████████████████████████████████| 368kB 57.1MB/s 
     |████████████████████████████████| 655kB 45.0MB/s 
     |████████████████████████████████| 194kB 41.4MB/s 
     |████████████████████████████████| 983kB 35.0MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 512kB 55.4MB/s 
     |████████████████████████████████| 3.8MB 53.1MB/s 


In [3]:
!sudo apt-get install gcc python-dev python-setuptools libffi-dev
!sudo apt-get install python-pip
!sudo pip install gsutil

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
gcc set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  python-pkg-resources
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  libffi-dev python-pkg-resources python-setuptools
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 613 kB of archives.
After this operation, 2,336 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-pkg-resources all 39.0.1-2 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-setuptools all 39.0.1-2 [329 kB]
Get:3 http://archive.ubuntu.com/ubuntu 

In [5]:
!mkdir ckpt
!gsutil cp -r gs://pegasus_ckpt ckpt/

mkdir: cannot create directory ‘ckpt’: File exists
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model...
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.vocab...
Copying gs://pegasus_ckpt/checkpoint...
Copying gs://pegasus_ckpt/model.ckpt-1500000.data-00000-of-00001...
\ [4 files][  2.1 GiB/  2.1 GiB]   95.0 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://pegasus_ckpt/model.ckpt-1500000.index...
Copying gs://pegasus_ckpt/model.ckpt-1500000.meta...
Copying gs://pegasus_ckpt/aeslc/model.ckpt-32000.data-00000-of-00001...
Copying gs://pegasus_ckpt/aeslc/model.ckpt-32000.index...
Copying gs://pegasus_ckpt/aeslc/model.ckpt-32000.meta...
Copying gs://pegasus_ckpt/arxiv/model.ckpt-340000.data-00000-of-00001...
Copying g

# New Section

In [6]:
%cd /content/
!pip3 install -e pegasus
%cd /content/pegasus/

/content
Obtaining file:///content/pegasus
  Running setup.py develop for pegasus
/content/pegasus


In [7]:
import pandas as pd
import tensorflow as tf

save_path = "/content/pegasus/pegasus/data/testdata/test_pattern_1.tfrecord"

input_dict = dict(
                  inputs=[ # here is the input text that goes in
                          "Musk’s current problems are less dire than those he faced in 2008, but Tesla and SpaceX are no longer experiments. Failure now would put thousands of jobs and billions of dollars at risk, and SpaceX has become crucial not just to the U.S. space program but also to countries and companies around the world hoping to put up satellites. Musk's supporters say he has built up enough credibility and has enough star power to raise money with relative ease or to be bailed out by one of his many ultrarich friends who share his aspirations. Just on technological merits, Musk’s companies have already changed the world. To the extent that the electric car is a commercial reality, it’s in large part thanks to Tesla. While it receives less attention, SpaceX and the revitalization of the U.S. aerospace industry may be Musk’s most stirring accomplishment. But at every point where his companies seem to be on stable footing, Musk takes on more and promises more, erasing the memory of past gains. The question is whether Musk can continue to operate on the edge of what’s possible — the key to his achievements so far."
                         ],
                  targets=[
                           "" # passing empty string since my target goal is to summerize the text
                          ]
                 )

data = pd.DataFrame(input_dict)

with tf.io.TFRecordWriter(save_path) as writer:
    for row in data.values:
        inputs, targets = row[:-1], row[-1]
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    "inputs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs[0].encode('utf-8')])),
                    "targets": tf.train.Feature(bytes_list=tf.train.BytesList(value=[targets.encode('utf-8')])),
                }
            )
        )
        writer.write(example.SerializeToString())

In [ ]:
!python3 pegasus/bin/train.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/aeslc



W0830 20:32:19.182901 139933262464896 module_wrapper.py:139] From /content/pegasus/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0830 20:32:19.388477 139933262464896 module_wrapper.py:139] From /content/pegasus/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0830 20:32:19.389236 139933262464896 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7f444499cb70>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'ckpt/pegasus_ckpt/aeslc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': N

In [ ]:
!python3 pegasus/bin/train.py --params=cnn_dailymail_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail



W0830 20:37:11.819154 140679300540288 module_wrapper.py:139] From /content/pegasus/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0830 20:37:12.039921 140679300540288 module_wrapper.py:139] From /content/pegasus/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0830 20:37:12.040601 140679300540288 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7ff1f7ef6b70>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'ckpt/pegasus_ckpt/cnn_dailymail', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_devic

In [ ]:
!python3 pegasus/bin/train.py --params=gigaword_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/gigaword



W0830 20:48:17.926240 140123635914624 module_wrapper.py:139] From /content/pegasus/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0830 20:48:18.144308 140123635914624 module_wrapper.py:139] From /content/pegasus/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0830 20:48:18.145047 140123635914624 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7f7097be2b70>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'ckpt/pegasus_ckpt/gigaword', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn'

In [ ]:
!python3 pegasus/bin/train.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu_long



W0830 22:35:22.670966 140449568884608 module_wrapper.py:139] From /content/pegasus/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0830 22:35:22.858121 140449568884608 module_wrapper.py:139] From /content/pegasus/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0830 22:35:22.858817 140449568884608 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7fbc7adc8b70>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'ckpt/pegasus_ckpt/reddit_tifu_long', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_de

In [ ]:
len("Musk’s current problems are less dire than those he faced in 2008, but Tesla and SpaceX are no longer experiments. Failure now would put thousands of jobs and billions of dollars at risk, and SpaceX has become crucial not just to the U.S. space program but also to countries and companies around the world hoping to put up satellites. Musk's supporters say he has built up enough credibility and has enough star power to raise money with relative ease or to be bailed out by one of his many ultrarich friends who share his aspirations. Just on technological merits, Musk’s companies have already changed the world. To the extent that the electric car is a commercial reality, it’s in large part thanks to Tesla. While it receives less attention, SpaceX and the revitalization of the U.S. aerospace industry may be Musk’s most stirring accomplishment. But at every point where his companies seem to be on stable footing, Musk takes on more and promises more, erasing the memory of past gains. The question is whether Musk can continue to operate on the edge of what’s possible — the key to his achievements so far.")

1113

In [ ]:
!python3 pegasus/bin/evaluate.py --params=test_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=1,beam_size=5,beam_alpha=0.6 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail



Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0831 00:52:16.574285 140705774061440 deprecation.py:323] From pegasus/bin/evaluate.py:85: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.

W0831 00:52:16.581033 140705774061440 module_wrapper.py:139] From /content/pegasus/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0831 00:52:16.809791 140705774061440 module_wrapper.py:139] From /content/pegasus/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0831 00:52:16.810527 140705774061440 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7ff821e19d90>) includes params argument, but params are not passe

In [ ]:
%load_ext tensorboard

In [ ]:
!python3 pegasus/bin/train.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu_long

In [ ]:
!python3 pegasus/bin/train.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu_long

In [ ]:
!python3 pegasus/bin/train.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu_long

In [ ]:
!python3 pegasus/bin/train.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/reddit_tifu_long

In [ ]:
!python3 pegasus/bin/train.py --params=newsroom_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/newsroom



W0830 20:37:11.819154 140679300540288 module_wrapper.py:139] From /content/pegasus/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0830 20:37:12.039921 140679300540288 module_wrapper.py:139] From /content/pegasus/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0830 20:37:12.040601 140679300540288 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7ff1f7ef6b70>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'ckpt/pegasus_ckpt/cnn_dailymail', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_devic

In [ ]:
!python3 pegasus/bin/evaluate.py --params=reddit_tifu_long_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=1,beam_size=5,beam_alpha=0.6 \
--model_dir=ckpt/pegasus_ckpt/aeslc



Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0830 20:31:40.234149 140633733166976 deprecation.py:323] From pegasus/bin/evaluate.py:85: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.

W0830 20:31:40.240438 140633733166976 module_wrapper.py:139] From /content/pegasus/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0830 20:31:40.465881 140633733166976 module_wrapper.py:139] From /content/pegasus/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0830 20:31:40.466556 140633733166976 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7fe75be8bc80>) includes params argument, but params are not passe

@registry.register("reddit_tifu_long_transformer")
def reddit_tifu_long_transformer(param_overrides):
  return transformer_params(
      {
          "train_pattern": "tfds_transformed:reddit_tifu/long-train",
          "dev_pattern": "tfds_transformed:reddit_tifu/long-validation",
          "test_pattern": "tfds_transformed:reddit_tifu/long-test",
          "max_input_len": 1024,
          "max_output_len": 128,
          "train_steps": 8000,
          "learning_rate": 0.0001,
          "batch_size": 8,
      }, param_overrides)

In [ ]:
#from google.colab import drive
#drive.mount("/content/drive/", force_remount=True)

In [ ]:
!python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=1,beam_size=5,beam_alpha=0.6 \
--model_dir=ckpt/pegasus_ckpt/aeslc